In [8]:
# #Using torch for the inverse kinematics
# import torch
# import numpy as np

# #Constant 
# torch.pi = torch.acos(torch.zeros(1)).item() * 2

# #Utiles functions 
# def point_to_rad(point_x, point_y):
#     """
#     Converts 2D cartesian points to polar angles and to be in the range of 0 to 2pi

#     Args:
#     point_x (tensor): x-coordinate
#     point_y (tensor): y-coordinate

#     Return:
#     angle (tensor): angle in radians

#     """

#     angle = torch.atan2(point_y, point_x)

#     #Adjust angle to be in the range 0 to 2pi
#     if angle < 0:
#         angle += 2 * torch.pi
#     return angle



# def rot_Matrix3d(rotation=[0, 0, 0], order='xyz'):

#     """
#     This function computes the rotation matrix for a 3D rotation given the rotation angles and the order of rotation
    

#     Args: 
#     rotation (list): list of the rotation angles in radians
#     order (str): order of rotation

#     Return:
#     rotationMatrix (tensor): rotation
#     """
#     roll, pitch, yaw = rotation 

#     #Rotation matrices for each axis
#     Rx = torch.tensor([[1, 0, 0], [0, torch.cos(roll), -torch.sin(roll)], [0, torch.sin(roll), torch.cos(roll)]])
#     Ry = torch.tensor([[torch.cos(pitch), 0, torch.sin(pitch)], [0, 1, 0], [-torch.sin(pitch), 0, torch.cos(pitch)]])
#     Rz = torch.tensor([[torch.cos(yaw), -torch.sin(yaw), 0], [torch.sin(yaw), torch.cos(yaw), 0], [0, 0, 1]])

#     #Dictionary to map order to rotation matrix multiplication
#     order_to_matrix = {
#         'xyz': lambda Rx, Ry, Rz: Rz @ Ry @ Rx,
#         'xzy': lambda Rx, Ry, Rz: Ry @ Rz @ Rx,
#         'yxz': lambda Rx, Ry, Rz: Rz @ Rx @ Ry,
#         'yzx': lambda Rx, Ry, Rz: Rx @ Rz @ Ry,
#         'zxy': lambda Rx, Ry, Rz: Ry @ Rx @ Rz,
#         'zyx': lambda Rx, Ry, Rz: Rx @ Ry @ Rz,
#     }

#     return order_to_matrix[order](Rx, Ry, Rz)
    

In [9]:
# #Testing block to chekc the correctness of the utiles functions
# #Test point_to_rad
# point_x = torch.tensor(1)
# point_y = torch.tensor(1)

# angle = point_to_rad(point_x, point_y)
# print(angle)

# #Test rot_Matrix3d
# rotation = [torch.tensor(0), torch.tensor(0), torch.tensor(0)]
# order = 'xyz'

# rotationMatrix = rot_Matrix3d(rotation, order)
# print(rotationMatrix)

In [10]:
# #Inverse Kinematic class for the robot

# """
# Important Notes:

# Default difinition for the four legs in Unitree Go1

#     Leg0 FR = right front leg
#     Leg1 FL = left front leg
#     Leg2 RR = right rear leg
#     Leg3 RL = left rear leg

# """
# import torch


# class InverseKinematics:

#     def __init__(self, robot_type='go1'):

#         # Define the robot type
#         self.robot_type = robot_type

#         # Define the robot parameters
#         if robot_type == 'go1':
#             # Leg IDs
#             self.FR = 0
#             self.FL = 1
#             self.RR = 2
#             self.RL = 3

#             # Robot parameters(meters):
#             self.hip_length = 0.08  # The distance between the hip joint and the thigh joint
#             self.thigh_length = 0.213
#             self.calf_length = 0.213
#             self.base_length = 0.3762
#             self.base_width = 0.0935
#             self.base_height = 0.0

#             # Leg origins(FR, FL, RR, RL)
#             self.leg_origins = torch.tensor([[self.base_length / 2, -self.base_width / 2, self.base_height],
#                                              [self.base_length / 2, self.base_width / 2, self.base_height],
#                                              [-self.base_length / 2, -self.base_width / 2, self.base_height],
#                                              [-self.base_length / 2, self.base_width / 2, self.base_height]])

#             # Define the right legs
#             self.right_legs = [self.FR, self.RR]

#     def calculation(self, xyz, right_legs):
#         # Unpack the foot's xyz coordinates.
#         x, y, z = xyz[0], xyz[1], xyz[2]

#         length_A = torch.norm(torch.tensor([y, z]))

#         a_2 = torch.arcsin(torch.sin(torch.pi / 2) * self.hip_length / length_A)

#         a_3 = torch.pi - a_2 - torch.pi / 2

#         a_1 = point_to_rad(y, z)

#         if right_legs:
#             theta_1 = a_1 - a_3
#         else:
#             theta_1 = a_1 + a_3
#             if theta_1 >= 2 * torch.pi:
#                 theta_1 -= 2 * torch.pi

#         # Hip Joint Coordinates
#         Joint_1 = torch.tensor([0, 0, 0])

#         # Thigh Joint Coordinates
#         Joint_2 = torch.tensor([0, self.hip_length * torch.cos(theta_1), self.hip_length * torch.sin(theta_1)])

#         # Foot Joint Coordinates
#         Joint_4 = torch.tensor([x, y, z])

#         # Vector from Joint_2 to Joint_4
#         Joint_4_2 = Joint_4 - Joint_2

#         if right_legs:
#             R = theta_1 - torch.pi
#         else:
#             R = theta_1

#         # Create a rotation matrix to work on a new 2D plane(XZ)
#         rotation_matrix = rot_Matrix3d([-R, 0, 0], order='xyz')
#         Joint_4_2_ = rotation_matrix @ Joint_4_2.unsqueeze(1)

#         x_, y_, z_ = Joint_4_2_[0][0], Joint_4_2_[1][0], Joint_4_2_[2][0]  # Fix: Access the elements of Joint_4_2_ correctly


#         length_B = torch.norm(torch.tensor([x_, z_]))

#         if length_B >= (self.thigh_length + self.calf_length):
#             length_B = (self.thigh_length + self.calf_length) * 0.99999
#             print('Target coordinate: [%f %f %f] too far away' % (x, y, z))

#         b_1 = point_to_rad(x_, z_)
#         b_2 = torch.acos((self.thigh_length**2 + length_B**2 - self.calf_length**2) / (2 * self.thigh_length * length_B))
#         b_3 = torch.acos((self.thigh_length**2 + self.calf_length**2 - length_B**2) / (2 * self.thigh_length * self.calf_length))

#         theta_2 = b_1 - b_2
#         theta_3 = torch.pi - b_3

#         #Calculate the joint coordinates for visualization

#         Joint_3_ = torch.tensor([self.thigh_length*torch.cos(theta_2), 0, self.thigh_length*torch.sin(theta_2)])
#         Joint_3 = Joint_2 + (rotation_matrix.inverse() @ Joint_3_.unsqueeze(1)).squeeze()

#         Joint_4_ = Joint_3_ + torch.tensor([self.calf_length*torch.cos(theta_2 + theta_3), 0, self.calf_length*torch.sin(theta_2 + theta_3)])
#         Joint_4 = Joint_2 + (rotation_matrix.inverse() @ Joint_4_.unsqueeze(1)).squeeze()

#         #Modify angles to match the robot's configuration
#         angles = self.angle_corrector([theta_1, theta_2, theta_3], right_legs)

#         return [angles[0], angles[1], angles[2], Joint_1, Joint_2, Joint_3, Joint_4]
    
#     def leg_IK(self, xyz, rot=[0, 0, 0], lenID =0, center_offset = [0, 0, 0]):

#         right_legs = lenID in self.right_legs

#         XYZ = 

#         pass 

#         return
    




#     def leg_IK(self, xyz, rot=[0, 0, 0], legID=0, is_radians=True, center_offset=[0, 0, 0]):
#         # Ensure inputs are tensors
#         xyz = torch.tensor(xyz, dtype=torch.float32)
#         rot = torch.tensor(rot, dtype=torch.float32)
#         center_offset = torch.tensor(center_offset, dtype=torch.float32)

#         # Check if the leg is from the right side
#         is_right = (legID in self.right_legs)
        
#         # Add offset of each leg from the axis of rotation
#         # Note: Assuming RotMatrix3D returns a tensor. If not, it needs adjustment.
#         rotation_matrix = RotMatrix3D(rot, is_radians)
#         XYZ = torch.inverse(rotation_matrix) @ (xyz + self.leg_origins[legID,:] - center_offset).T
        
#         # Subtract the offset between the leg and the center of rotation
#         # so that the resultant coordinate is relative to the origin (j1) of the leg
#         xyz_ = (XYZ.T - self.leg_origins[legID,:] + center_offset).flatten()

#         # Calculate the angles and coordinates of the leg relative to the origin of the leg
#         return self.leg_IK_calc(xyz_, is_right)


#     def angle_corrector(self, angles, right_legs=True):

#         angles[1] -= 1.5*torch.pi

#         if right_legs:
#             theta_1 = angles[0] - torch.pi
#             theta_2 = angles[1] + 0*torch.pi/180

#         else:
#             if angles[0] > torch.pi:
#                 theta_1 = angles[0] - 2*torch.pi

#             else:
#                 theta_1 = angles[0]

#             theta_2 = -angles[1] - 0*torch.pi/180

#         theta_3 = -angles[2] + 0*torch.pi/180

#         return [theta_1, theta_2, theta_3]





In [11]:
# kin = InverseKinematics(robot_type='go1')

# foot_positions = [
#     [0, -0.08, -0.35],  # FR
#     [0, 0.08, -0.35],  # LF
#     [0, -0.08, -0.35],  # RR
#     [0, 0.08, -0.35],  # RL
# ]


# for i, foot_position in enumerate(foot_positions):
#     angles = kin.leg_IK(foot_position, legID=i)
#     print(f"Leg {i} angles: Hip: {angles[0]}, Thigh: {angles[1]}, Calf: {angles[2]}")


In [12]:


from math import atan, pi, radians, cos, sin
import numpy as np
from numpy.linalg import inv, norm
from numpy import array, asarray, matrix
from math import *
import matplotlib.pyplot as plt



# def point_to_rad(p1, p2): # converts 2D cartesian points to polar angles in range 0 - 2pi
        
#     if (p1 > 0 and p2 >= 0): return atan(p2/(p1))
#     elif (p1 == 0 and p2 >= 0): return pi/2
#     elif (p1 < 0 and p2 >= 0): return -abs(atan(p2/p1)) + pi
#     elif (p1 < 0 and p2 < 0): return atan(p2/p1) + pi
#     elif (p1 > 0 and p2 < 0): return -abs(atan(p2/p1)) + 2*pi
#     elif (p1 == 0 and p2 < 0): return pi * 3/2
#     elif (p1 == 0 and p2 == 0): return pi * 3/2 # edge case


from math import atan2, pi

def point_to_rad(p1, p2):
    # Converts 2D cartesian points to polar angles in range -pi to pi
    angle = atan2(p2, p1)
    
    # Adjust angle to be in the range 0 to 2pi
    if angle < 0:
        angle += 2*pi
    
    return angle
    
def RotMatrix3D(rotation=[0,0,0],is_radians=True, order='xyz'):
    
    roll, pitch, yaw = rotation[0], rotation[1], rotation[2]

    # convert to radians is the input is in degrees
    if not is_radians: 
        roll = radians(roll)
        pitch = radians(pitch)
        yaw = radians(yaw)
    
    # rotation matrix about each axis
    rotX = np.matrix([[1, 0, 0], [0, cos(roll), -sin(roll)], [0, sin(roll), cos(roll)]])
    rotY = np.matrix([[cos(pitch), 0, sin(pitch)], [0, 1, 0], [-sin(pitch), 0, cos(pitch)]])
    rotZ = np.matrix([[cos(yaw), -sin(yaw), 0], [sin(yaw), cos(yaw), 0], [0, 0, 1]])
    
    # rotation matrix order (default: pitch -> roll -> yaw)
    if order == 'xyz': rotationMatrix = rotZ * rotY * rotX
    elif order == 'xzy': rotationMatrix = rotY * rotZ * rotX
    elif order == 'yxz': rotationMatrix = rotZ * rotX * rotY
    elif order == 'yzx': rotationMatrix = rotX * rotZ * rotY
    elif order == 'zxy': rotationMatrix = rotY * rotX * rotZ
    elif order == 'zyx': rotationMatrix = rotX * rotY * rotZ
    
    return rotationMatrix # roll pitch and yaw rotation 


class kinematics():
    
    def __init__(self):
        
        # note: leg IDs
        left_front = 0
        left_back  = 1
        right_front= 2
        right_back = 3
        
        self.right_legs = [right_front, right_back]
        
        self.link_1 = 0.08
        self.link_2 = 0.213
        self.link_3 = 0.213
        self.phi = radians(90) 
        
        # body dimensions
        self.length = 0.3762
        self.width = 0.0935
        self.hight = 0.0
        
        # leg origins (left_f, left_b, right_b, right_f), i.e., the coordinate of j1
        self.leg_origins = np.matrix([[self.length/2, self.width/2, 0],
                          [-self.length/2, self.width/2, 0],
                          [-self.length/2, -self.width/2, 0],
                          [self.length/2, -self.width/2, 0],
                          [self.length/2, self.width/2, 0]])
        
    # this method adjust inputs to the IK calculator by adding rotation and 
    # offset of that rotation from the center of the robot
    def leg_IK(self, xyz, rot = [0,0,0], legID=0, is_radians=True, center_offset=[0,0,0]):

        #what is xyz? what does it represent? 
        
        # check is the leg is from the right side 
        is_right = (legID in self.right_legs)
        
        # add offset of each leg from the axis of rotation
        XYZ = asarray((inv(RotMatrix3D(rot,is_radians)) * \
            ((array(xyz) + self.leg_origins[legID,:] - array(center_offset)).transpose())).transpose())
        
        # subtract the offset between the leg and the center of rotation 
        # so that the resultant coordiante is relative to the origin (j1) of the leg
        xyz_ = asarray(XYZ - self.leg_origins[legID,:] + array(center_offset)).flatten()

        # calculate the angles and coordinates of the leg relative to the origin of the leg
        return self.leg_IK_calc(xyz_, is_right)


    # IK calculator
    def leg_IK_calc(self, xyz, is_right=False): 

        #Get the foot x, y, z coordinates
        x, y, z = xyz[0], xyz[1], xyz[2] 
        
        # length of vector projected on the YZ plane. equiv. to len_A = sqrt(y**2 + z**2)
        len_A = np.sqrt(y**2 + z**2)

        #Calculate angle between len_A and the leg's projection line on YZ plane
        a_2 = np.arcsin(np.sin(self.phi)*self.link_1/len_A)

        #Calculate angle between link_1 and length len_A
        a_3 = np.pi - a_2 - self.phi

        #Calculate the angle from the positive y-axis to the end-effector
        a_1 = point_to_rad(y,z)

        #Length of the leg project in the YZ plane
        # len_H = np.sqrt(len_A**2 -self.link_1**2) 

        # angle_1 = np.arctan2(len_A, y)
        # theta_1 = a_3 - angle_1

        #Angle of link1 about the x-axis in positive y-axis
        if is_right: 
            theta_1 = a_1 - a_3
        else: 
            theta_1 = a_1 + a_3

            if theta_1 >= 2*np.pi:
                theta_1 -= 2*np.pi

        
        #Hip joint coordinates
        j1 = array([0,0,0]) 
        
        #Thight joint coordinates 
        #x, y, z = j2[0], j2[1], j2[2]
        j2 = array([0,self.link_1*cos(theta_1),self.link_1*sin(theta_1)]) 

        #Foot joint coordinates
        j4 = array(xyz)
        #Vector from j2 to j4, i.e., the vector from the thight to the foot
        j4_2_vec = j4 - j2
    
        
        if is_right:
            R = theta_1 - self.phi - pi/2
        else: 
            R = theta_1 + self.phi - pi/2
        
        # create rotation matrix to work on a new 2D plane (XZ_)
        rot_mtx = RotMatrix3D([-R,0,0],is_radians=True)
        j4_2_vec_ = rot_mtx * (np.reshape(j4_2_vec,[3,1]))
        
        # xyz in the rotated coordinate system + offset due to link_1 removed
        x_, y_, z_ = j4_2_vec_[0], j4_2_vec_[1], j4_2_vec_[2]
        
        len_B = norm([x_, z_]) # norm(j4-j2)
        
        # handling mathematically invalid input, i.e., point too far away to reach
        if len_B >= (self.link_2 + self.link_3): 
            len_B = (self.link_2 + self.link_3) * 0.99999
            # self.node.get_logger().warn('target coordinate: [%f %f %f] too far away' % (x, y, z))
            print('target coordinate: [%f %f %f] too far away' % (x, y, z))
        
        # b_1 : angle between +ve x-axis and len_B (0 <= b_1 < 2pi)
        # b_2 : angle between len_B and link_2
        # b_3 : angle between link_2 and link_3
        b_1 = point_to_rad(x_, z_)  
        b_2 = acos((self.link_2**2 + len_B**2 - self.link_3**2) / (2 * self.link_2 * len_B)) 
        b_3 = acos((self.link_2**2 + self.link_3**2 - len_B**2) / (2 * self.link_2 * self.link_3))  
        
        # assuming theta_2 = 0 when the leg is pointing down (i.e., 270 degrees offset from the +ve x-axis)
        theta_2 = b_1 - b_2    
        theta_3 = pi - b_3
        
        # CALCULATE THE COORDINATES OF THE JOINTS FOR VISUALIZATION
        j1 = np.array([0,0,0])
        
        # calculate joint 3
        j3_ = np.reshape(np.array([self.link_2*cos(theta_2),0, self.link_2*sin(theta_2)]),[3,1])
        j3 = np.asarray(j2 + np.reshape(np.linalg.inv(rot_mtx)*j3_, [1,3])).flatten()
        
        # calculate joint 4
        j4_ = j3_ + np.reshape(np.array([self.link_3*cos(theta_2+theta_3),0, self.link_3*sin(theta_2+theta_3)]), [3,1])
        j4 = np.asarray(j2 + np.reshape(np.linalg.inv(rot_mtx)*j4_, [1,3])).flatten()
        
        # modify angles to match robot's configuration (i.e., adding offsets)
        angles = self.angle_corrector(angles=[theta_1, theta_2, theta_3], is_right=is_right)
        #angles = [theta_1, theta_2, theta_3]
        # print(degrees(angles[0]))
        return [angles[0], angles[1], angles[2], j1, j2, j3, j4]
    
    # TO-DO : modify this function depending on your robot's configuration
    # adjusting angle for specific configurations of motors, incl. orientation
    # this will vary for each robot (possibly for each leg as well)
    def angle_corrector(self, angles=[0,0,0], is_right=True):
        angles[1] -= 1.5*pi; # add offset # 90 degrees
        if is_right:
            theta_1 = angles[0] - pi
            theta_2 = angles[1] + 0*pi/180 # 45 degrees initial offset #
        else: 
            if angles[0] > pi:  
                theta_1 = angles[0] - 2*pi
            else: theta_1 = angles[0]
            
            theta_2 = -angles[1] - 0*pi/180
        
        theta_3 = -angles[2] + 0*pi/180
        return [theta_1, theta_2, theta_3]


In [14]:
k = kinematics()

foot_positions = [
    [0, 0.08, -0.35],  # Front-left
    [0, 0.08, -0.35],  # Front-right
    [0, -0.08, -0.35],  # Rear-left
    [0, -0.08, -0.35],  # Rear-right
]

for i, foot_position in enumerate(foot_positions):
    angles = k.leg_IK(foot_position, legID=i)
    print(f"Leg {i} angles: Hip: {angles[0]}, Thigh: {angles[1]}, Calf: {angles[2]}")

    # print(f" \n Leg {i} angles: Hip: {angles[0] * 180 / pi}, Thigh: {angles[1] * 180 / pi}, Calf: {angles[2] * 180 / pi}")
    

Leg 0 angles: Hip: 0.0, Thigh: 0.6065908463830638, Calf: -1.2131816927661283
Leg 1 angles: Hip: 0.0, Thigh: 0.6065908463830638, Calf: -1.2131816927661283
Leg 2 angles: Hip: -4.440892098500626e-16, Thigh: -0.6065908463830638, Calf: -1.2131816927661276
Leg 3 angles: Hip: -4.440892098500626e-16, Thigh: -0.6065908463830638, Calf: -1.2131816927661276
